In [1]:
import pandas as pd
import numpy as np
file_path=r'C:\Users\ratha\Downloads\Copy of RBI_Data.xlsx'

#Read the excel file
df=pd.read_excel(file_path)

#Code for displaying all the columns and rows of the dataset
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)

#Display the dataset
print(df)

           year  Currency in circulation -Total  `Other' deposits with RBI   \
0    2020-08-21                     2.688322e+06                 39768.5731   
1    2020-08-14                     2.691706e+06                 39542.5731   
2    2020-08-07                     2.686070e+06                 39754.5731   
3    2020-07-31                     2.672446e+06                 39706.5731   
4    2020-07-24                     2.670987e+06                 39206.5731   
5    2020-07-17                     2.675760e+06                 39275.5731   
6    2020-07-10                     2.677781e+06                 39728.2731   
7    2020-07-03                     2.665329e+06                 40178.2731   
8    2020-06-26                     2.660382e+06                 39295.6834   
9    2020-06-19                     2.660533e+06                 38715.6834   
10   2020-06-12                     2.658733e+06                 39198.6834   
11   2020-06-05                     2.639322e+06    

In [3]:
#Removing the the row with 967 index due to too many missing values in the row
df.drop(index=967, inplace=True)
columns_with_dash = df.columns[(df == '-').any()].tolist()

# Display the columns containing '-'
print("Columns with '-' values:", columns_with_dash)

Columns with '-' values: ["RBI's Claims on - Central Govt"]


In [5]:
#Replacing the '-' i.e. the misssing values with NaN
df["RBI's Claims on - Central Govt"] = df["RBI's Claims on - Central Govt"].replace('-', np.nan)

C:\Users\ratha\AppData\Local\Temp\ipykernel_20460\1945819632.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["RBI's Claims on - Central Govt"] = df["RBI's Claims on - Central Govt"].replace('-', np.nan)


In [7]:
from sklearn.mixture import GaussianMixture as GMM

#EM algorithm is used to estimate the missing values in the column 'RBI's Claims on - Central Govt'
pd.set_option('future.no_silent_downcasting', True)
def em_impute(df):
    # Create a mask for missing values
    mask = df.isnull()
    numeric_column=["RBI's Claims on - Central Govt"]
    # Create an initial guess for the missing values
    imputed_df=df.copy()
    imputed_df[numeric_column] = imputed_df[numeric_column].fillna(imputed_df[numeric_column].mean())
    
    # Initialize the GMM model
    gmm = GMM(n_components=1, random_state=0)

    # Iterate until convergence
    for _ in range(10):  # You can adjust the number of iterations
        # Fit the model to the complete cases
        gmm.fit(imputed_df[numeric_column].dropna())

        # Predict the missing values
        for col in numeric_column:
            if df[col].isnull().any():
                missing_data = imputed_df[mask[col]]
                imputed_values = gmm.sample(missing_data.shape[0])[0]
                imputed_df.loc[mask[col], col] = imputed_values[:, 0]

    return imputed_df

# Impute missing values
imputed_data = em_impute(df)
print("Imputed Data:\n", imputed_data)

C:\Users\ratha\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ratha\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ratha\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ratha\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Window

Imputed Data:
            year  Currency in circulation -Total  `Other' deposits with RBI   \
0    2020-08-21                     2.688322e+06                 39768.5731   
1    2020-08-14                     2.691706e+06                 39542.5731   
2    2020-08-07                     2.686070e+06                 39754.5731   
3    2020-07-31                     2.672446e+06                 39706.5731   
4    2020-07-24                     2.670987e+06                 39206.5731   
5    2020-07-17                     2.675760e+06                 39275.5731   
6    2020-07-10                     2.677781e+06                 39728.2731   
7    2020-07-03                     2.665329e+06                 40178.2731   
8    2020-06-26                     2.660382e+06                 39295.6834   
9    2020-06-19                     2.660533e+06                 38715.6834   
10   2020-06-12                     2.658733e+06                 39198.6834   
11   2020-06-05                     2

In [46]:
output_file_path = 'cleaned_data_2.xlsx'

# Save the cleaned DataFrame to an Excel file
imputed_data.to_excel(output_file_path, index=False)